In [1]:
from main import DataImporter
from models.modelhelper import ModelHelper
import numpy as np
import pandas as pd

data_importer = DataImporter()
data_importer.import_data()

ds_train = data_importer.get_train_data()
ds_test = data_importer.get_test_data()
ds_validation = data_importer.get_validation_data()

In [2]:
ds_train.info()
print('\n')
ds_test.info()
print('\n')
ds_validation.info()
print('\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10269 entries, 0 to 10268
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10269 non-null  object 
 1   label                 10269 non-null  int64  
 2   statement             10269 non-null  object 
 3   subject               10269 non-null  object 
 4   speaker               10269 non-null  object 
 5   job_title             10269 non-null  object 
 6   state_info            10269 non-null  object 
 7   party_affiliation     10269 non-null  object 
 8   barely_true_counts    10269 non-null  float32
 9   false_counts          10269 non-null  float32
 10  half_true_counts      10269 non-null  float32
 11  mostly_true_counts    10269 non-null  float32
 12  pants_on_fire_counts  10269 non-null  float32
 13  context               10269 non-null  object 
dtypes: float32(5), int64(1), object(8)
memory usage: 922.7+ KB


<class 'p

In [3]:
# Drop specified columns from each dataset
columns_to_drop = ['id', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'context']

ds_train = ds_train.drop(columns=columns_to_drop)
ds_test = ds_test.drop(columns=columns_to_drop) 
ds_validation = ds_validation.drop(columns=columns_to_drop)

# Display results
print("Training Dataset:")
display(ds_train.head(50))
print("\nTest Dataset:") 
display(ds_test.head(50))
print("\nValidation Dataset:")
display(ds_validation.head(50))



Training Dataset:


,label,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts
0,0,Says the Annies List political group supports ...,0.0,1.0,0.0,0.0,0.0
1,1,When did the decline of coal start? It started...,0.0,0.0,1.0,1.0,0.0
2,2,"Hillary Clinton agrees with John McCain ""by vo...",70.0,71.0,160.0,163.0,9.0
3,0,Health care reform legislation is likely to ma...,7.0,19.0,3.0,5.0,44.0
4,1,The economic turnaround started at the end of ...,15.0,9.0,20.0,19.0,2.0
5,3,The Chicago Bears have had more starting quart...,0.0,3.0,2.0,5.0,1.0
6,4,Jim Dunnam has not lived in the district he re...,3.0,1.0,1.0,3.0,1.0
7,1,"""I'm the only person on this stage who has wor...",70.0,71.0,160.0,163.0,9.0
8,1,"However, it took $19.5 million in Oregon Lotte...",0.0,0.0,1.0,0.0,1.0
9,2,Says GOP primary opponents Glenn Grothman and ...,0.0,0.0,0.0,1.0,0.0



Test Dataset:


,label,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts
0,3,Building a wall on the U.S.-Mexico border will...,30.0,30.0,42.0,23.0,18.0
1,0,Wisconsin is on pace to double the number of l...,2.0,1.0,0.0,0.0,0.0
2,0,Says John McCain has done nothing to help the ...,63.0,114.0,51.0,37.0,61.0
3,1,Suzanne Bonamici supports a plan that will cut...,1.0,1.0,3.0,1.0,1.0
4,5,When asked by a reporter whether hes at the ce...,5.0,7.0,2.0,2.0,7.0
5,3,Over the past five years the federal governmen...,1.0,2.0,1.0,1.0,0.0
6,3,Says that Tennessee law requires that schools ...,0.0,0.0,0.0,0.0,0.0
7,4,"Says Vice President Joe Biden ""admits that the...",13.0,22.0,11.0,4.0,2.0
8,3,Donald Trump is against marriage equality. He ...,0.0,0.0,0.0,0.0,0.0
9,4,We know that more than half of Hillary Clinton...,8.0,10.0,12.0,5.0,0.0



Validation Dataset:


,label,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts
0,4,We have less Americans working now than in the...,1.0,0.0,1.0,0.0,0.0
1,5,"When Obama was sworn into office, he DID NOT u...",11.0,43.0,8.0,5.0,105.0
2,0,Says Having organizations parading as being so...,0.0,1.0,1.0,1.0,0.0
3,1,Says nearly half of Oregons children are poor.,0.0,1.0,1.0,1.0,0.0
4,1,On attacks by Republicans that various program...,70.0,71.0,160.0,163.0,9.0
5,0,Says when armed civilians stop mass shootings ...,1.0,1.0,0.0,1.0,0.0
6,3,Says Tennessee is providing millions of dollar...,0.0,0.0,0.0,0.0,0.0
7,0,The health care reform plan would set limits s...,4.0,5.0,4.0,2.0,0.0
8,3,Says Donald Trump started his career back in 1...,40.0,29.0,69.0,76.0,7.0
9,1,Bill White has a long history of trying to lim...,3.0,1.0,1.0,3.0,1.0


In [4]:
def new_false(ds_train, ds_test, ds_validation):
    """
    Combines the 'false_counts' and 'pants_on_fire_counts' columns into a single 'false_counts' column,
    then drops the 'pants_on_fire_counts' column from each dataset.

    Parameters:
        ds_train (pd.DataFrame): Training dataset
        ds_test (pd.DataFrame): Test dataset
        ds_validation (pd.DataFrame): Validation dataset

    Returns:
        tuple: Updated (ds_train, ds_test, ds_validation) DataFrames
    """

    #Quick note. We need to combine the data for the two columns in the label column as well. This will make it so that we can get equal sample sizes for each label.

    # Combine false_counts and pants_on_fire_counts into a single 'false_counts' column
    for ds in [ds_train, ds_test, ds_validation]:
        ds['false_counts'] = ds['false_counts'] + ds['pants_on_fire_counts']

        ds.drop('pants_on_fire_counts', axis=1, inplace=True)
    return ds_train, ds_test, ds_validation

def merge_columns(df, col_to_merge, col_to_keep, merge_label_value, keep_label_value):
    """
    Merges one column's data into another and updates corresponding labels.
    
    Args:
        df (pd.DataFrame): Input DataFrame
        col_to_merge (str): Name of column to be merged
        col_to_keep (str): Name of column to merge into
        merge_label_value (int): Label value associated with col_to_merge 
        keep_label_value (int): Label value associated with col_to_keep
        
    Returns:
        pd.DataFrame: DataFrame with merged columns and updated labels
    """
    # Create copy to avoid modifying original
    df = df.copy()
    
    # Add values from col_to_merge into col_to_keep
    df[col_to_keep] = df[col_to_keep] + df[col_to_merge]
    
    # Update labels - change merge_label_value to keep_label_value
    df.loc[df['label'] == merge_label_value, 'label'] = keep_label_value
    
    # Drop the merged column
    df.drop(col_to_merge, axis=1, inplace=True)
    
    return df


#False = 0
#Half-True = 1
#Mostly-True / true = 3, 2
#Barely-True = 4
#Pants-On-Fire = 5

# Update label 2 to 3 in all datasets. 
# #This is because true and mostly true are the same thing and they were combined into 1 without updating the labes.
ds_train.loc[ds_train['label'] == 2, 'label'] = 3
ds_test.loc[ds_test['label'] == 2, 'label'] = 3 
ds_validation.loc[ds_validation['label'] == 2, 'label'] = 3

#Merge pants on fire counts into false counts
ds_train = merge_columns(ds_train, 'pants_on_fire_counts', 'false_counts', 5, 0)
ds_test = merge_columns(ds_test, 'pants_on_fire_counts', 'false_counts', 5, 0)
ds_validation = merge_columns(ds_validation, 'pants_on_fire_counts', 'false_counts', 5, 0)

#Merge barely true counts into false counts
ds_train = merge_columns(ds_train, 'barely_true_counts', 'false_counts', 4, 0)
ds_test = merge_columns(ds_test, 'barely_true_counts', 'false_counts', 4, 0)
ds_validation = merge_columns(ds_validation, 'barely_true_counts', 'false_counts', 4, 0)

display(ds_train.head())
display(ds_test.head())
display(ds_validation.head())



,label,statement,false_counts,half_true_counts,mostly_true_counts
0,0,Says the Annies List political group supports ...,1.0,0.0,0.0
1,1,When did the decline of coal start? It started...,0.0,1.0,1.0
2,3,"Hillary Clinton agrees with John McCain ""by vo...",150.0,160.0,163.0
3,0,Health care reform legislation is likely to ma...,70.0,3.0,5.0
4,1,The economic turnaround started at the end of ...,26.0,20.0,19.0


,label,statement,false_counts,half_true_counts,mostly_true_counts
0,3,Building a wall on the U.S.-Mexico border will...,78.0,42.0,23.0
1,0,Wisconsin is on pace to double the number of l...,3.0,0.0,0.0
2,0,Says John McCain has done nothing to help the ...,238.0,51.0,37.0
3,1,Suzanne Bonamici supports a plan that will cut...,3.0,3.0,1.0
4,0,When asked by a reporter whether hes at the ce...,19.0,2.0,2.0


,label,statement,false_counts,half_true_counts,mostly_true_counts
0,0,We have less Americans working now than in the...,1.0,1.0,0.0
1,0,"When Obama was sworn into office, he DID NOT u...",159.0,8.0,5.0
2,0,Says Having organizations parading as being so...,1.0,1.0,1.0
3,1,Says nearly half of Oregons children are poor.,1.0,1.0,1.0
4,1,On attacks by Republicans that various program...,150.0,160.0,163.0


# Getting Data and Model ready
## Load model_helper

In [5]:
# Initialize ModelHelper
model_helper = ModelHelper()

In [6]:
# Get balanced dataset by sampling equal number of records for each label
def balance_dataset(df):
    """
    Balance dataset by sampling equal number of records for each unique label.
    
    Args:
        df: Input DataFrame with 'label' column
        
    Returns:
        Balanced DataFrame with equal samples per label
    """
    # Find minimum count across labels
    min_count = df['label'].value_counts().min()
    
    # Sample equal number of records for each label
    balanced_dfs = []
    for label in df['label'].unique():
        label_df = df[df['label'] == label].sample(n=min_count, random_state=42)
        balanced_dfs.append(label_df)
    
    # Combine balanced samples
    return pd.concat(balanced_dfs, ignore_index=True)

# Balance each dataset
ds_train = balance_dataset(ds_train)
ds_test = balance_dataset(ds_test) 
ds_validation = balance_dataset(ds_validation)

# Display class distribution after balancing
print("\nClass distribution after balancing:")
print("\nTraining set:")
print(ds_train['label'].value_counts())
print("\nTest set:")
print(ds_test['label'].value_counts())
print("\nValidation set:") 
print(ds_validation['label'].value_counts())



Class distribution after balancing:

Training set:
label
0    2123
1    2123
3    2123
Name: count, dtype: int64

Test set:
label
3    267
0    267
1    267
Name: count, dtype: int64

Validation set:
label
0    248
1    248
3    248
Name: count, dtype: int64


## Create Vectorizor and start preparing data

In [7]:
#Create Vectorizer
model_helper.create_vectorizer(ds_train['statement'], max_sequence_length=60)

# Preprocess text data
train_sequences = model_helper.preprocess_text(ds_train['statement'].tolist())
test_sequences = model_helper.preprocess_text(ds_test['statement'].tolist())
val_sequences = model_helper.preprocess_text(ds_validation['statement'].tolist())

## Get Sequence Columns

In [8]:
# Get truthfulness columns
truthfulness_columns = model_helper.truthfulness_columns
truthfulness_columns.remove('pants_on_fire_counts')
truthfulness_columns.remove('barely_true_counts')


## Normalizing data

In [9]:
# Get the raw count values for training
train_labels = model_helper.normalize_counts(ds_train)
test_labels = model_helper.normalize_counts(ds_test)
val_labels = model_helper.normalize_counts(ds_validation)

## Create classification model

In [10]:
# Create text classification model
model = model_helper.create_text_classification_model(
    num_classes=len(truthfulness_columns)
)

## Prepare Datasets

In [11]:
# Prepare datasets
train_dataset, val_dataset, test_dataset = model_helper.prepare_datasets(
    train_sequences=train_sequences,
    train_labels=train_labels,
    val_sequences=val_sequences,
    val_labels=val_labels,
    test_sequences=test_sequences,
    test_labels=test_labels,
    batch_size=32
)

## Training the model

In [12]:
%load_ext tensorboard
%tensorboard --logdir models/logs/fit

# Train the model using ModelHelper's train_model method
history = model_helper.train_model(
    model=model,
    train_data=train_dataset,
    validation_data=val_dataset,
    epochs=300,
    batch_size=32
)

Reusing TensorBoard on port 6006 (pid 14640), started 6 days, 14:31:55 ago. (Use '!kill 14640' to kill it.)

Epoch 1/300

200/200 [==============================] - 2s 5ms/step - loss: 1.8665 - accuracy: 0.4447 - categorical_accuracy: 0.4447 - auc: 0.5709 - precision: 0.8450 - recall: 0.2485 - val_loss: 1.9203 - val_accuracy: 0.3616 - val_categorical_accuracy: 0.3616 - val_auc: 0.5231 - val_precision: 0.8066 - val_recall: 0.1629 - lr: 9.9902e-05
Epoch 2/300
200/200 [==============================] - 1s 4ms/step - loss: 1.7020 - accuracy: 0.4995 - categorical_accuracy: 0.4995 - auc: 0.6113 - precision: 0.8855 - recall: 0.2076 - val_loss: 1.8666 - val_accuracy: 0.3750 - val_categorical_accuracy: 0.3750 - val_auc: 0.5061 - val_precision: 0.7762 - val_recall: 0.1199 - lr: 9.9608e-05
Epoch 3/300
200/200 [==============================] - 1s 4ms/step - loss: 1.6395 - accuracy: 0.5298 - categorical_accuracy: 0.5298 - auc: 0.6344 - precision: 0.9022 - recall: 0.1995 - val_loss: 1.8083 - val_accuracy: 0.4140 - val_categorical_accuracy: 0.4140 - val_auc: 0.5045 - val_precision: 0.7788 - val_recall: 0.0

## Evaluate Model

In [18]:
# Evaluate on test set
test_metrics = model.evaluate(test_dataset)
print(test_metrics)

26/26 [==============================] - 0s 2ms/step - loss: 1.1198 - accuracy: 0.6742 - categorical_accuracy: 0.6742 - auc: 0.5183 - precision: 0.9135 - recall: 0.0498
[1.119826316833496, 0.6741573214530945, 0.6741573214530945, 0.5183225274085999, 0.9134615659713745, 0.049764275550842285]


## Save model

In [14]:
# Save the model
model_helper.save_model(model, "text_classification_model_final")

True

## Load and Use Saved model

In [15]:
# Load the saved model
loaded_model = model_helper.load_model("text_classification_model_final")
test_string = 'Barbara Buono by the numbers: As a Trenton politician, she voted to raise taxes 154 times. Under her, property taxes up 70 percent. Backed a 16 percent sales tax increase. Utilities, nursing homes, cell phones, parking lots, lottery wins, gyms She taxed them all. Architect of Corzines budget, she drove New Jersey $2 billion into debt. Barbara Buono by the numbers: taking New Jersey backwards.'
print(model_helper.preprocess_text(test_string))
prediction = loaded_model.predict(model_helper.preprocess_text(test_string))
print(prediction)

[[ 7.80787924e-03 -1.44528849e-02  6.46416424e-03 -2.56793685e-02
  -9.65327397e-02  8.58551189e-02  6.98082447e-02  3.99074377e-03
  -8.74180868e-02  2.87988503e-02  2.87528969e-02  1.01409713e-02
   2.93969605e-02 -8.59811381e-02 -3.12866308e-02  7.26936832e-02
   4.58125258e-03  8.62116888e-02  2.10175058e-03  1.01940013e-01
   7.18423799e-02  4.75919712e-03 -9.46561049e-05  5.05877845e-02
   1.56101644e-01  4.36919257e-02 -1.51563315e-02  2.40388163e-03
  -1.58895310e-02 -2.51775812e-02 -1.06737919e-01  7.07412809e-02
   4.94894832e-02  3.08563840e-02  2.91129984e-02 -3.40982117e-02
   1.11657195e-01  4.70892824e-02 -6.71625230e-03  2.94581391e-02
  -4.25266698e-02 -3.54633946e-03 -1.84288230e-02 -1.99071560e-02
  -8.40682983e-02 -8.74066129e-02  5.21774963e-02 -1.68546159e-02
   4.58840802e-02  2.17433758e-02 -6.76908568e-02  4.28052358e-02
  -4.56975177e-02 -2.15390436e-02  2.67212316e-02 -1.36472238e-02
   5.95129468e-03  8.63851160e-02  2.10052487e-02 -1.97168756e-02
   1.13614

## Use prepared test data to get Sample Predictions

In [16]:
# Make predictions on test dataset
print(test_dataset)
predictions = loaded_model.predict(test_dataset)
# Convert predictions to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Print sample predictions
print("\nSample predictions:")
for i in range(50):
    print(f"Example {i+1}:")
    print(f"Predicted probabilities: {predictions[i]}")
    print(f"Predicted class: {predicted_classes[i]}")
    print(f"Actual class: {np.argmax(test_labels[i])}\n")


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 384), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>
26/26 [==============================] - 0s 2ms/step

Sample predictions:
Example 1:
Predicted probabilities: [0.43882662 0.31933266 0.24184068]
Predicted class: 0
Actual class: 0

Example 2:
Predicted probabilities: [0.45658416 0.2490904  0.29432547]
Predicted class: 0
Actual class: 0

Example 3:
Predicted probabilities: [0.550378   0.17213032 0.27749166]
Predicted class: 0
Actual class: 0

Example 4:
Predicted probabilities: [0.47776264 0.31845286 0.20378453]
Predicted class: 0
Actual class: 0

Example 5:
Predicted probabilities: [0.4353775  0.261791   0.30283144]
Predicted class: 0
Actual class: 0

Example 6:
Predicted probabilities: [0.3908481  0.37031868 0.23883316]
Predicted class: 0
Actual class: 0

Example 7:
Predicted probabilities: [0.42929986 0.2669491  0.30375102]
Predicted class: 0
Actual class: 0

Example 8:
Predicted proba

In [17]:
models_list = model_helper.list_models()
print(models_list)

['text_classification_model', 'text_classification_model_final']
